In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets

iris = datasets.load_iris()

In [2]:
# define membership functions
def trapMem(x, params):
    mem = []
    a,b,c,d = np.r_[params]
    for val in x:
        if (a != b):
            term1 = (val - a) / (b - a)
        else:
            term1 = 1
        if (c != d):
            term2 = (d - val) / (d - c)
        else: 
            term2 = 1
        if (a - 0.00001 <= val <= d + 0.00001):
            value = max(min(term1, term2, 1), 0)
        else:
            value = 0
        mem.append(value)
        # print(val, value)
    return mem

def gaussMem(x, params):
    mem = []
    sigma, c = np.r_[params]
    for val in x:
        term1 = -1 * ((val - c) ** 2)
        term2 = 2 * (sigma ** 2)
        value = np.e ** (term1 / term2)
        mem.append(value)
        # print(val, value)
    return mem

def trigMem(x, params):
    mem = []
    a,b,c = np.r_[params]
    for val in x:
        if (a != b):
            term1 = (val - a) / (b - a)
        else:
            term1 = 1
        if (b != c):
            term2 = (c - val) / (c - b)
        else:
            term2 = 1
        if(val < a - 0.00001 or val > c + 0.00001):
            value = 0
        else:
            value = max(min(term1,term2), 0)
        mem.append(value)
        # print(val, value)
    return mem

def centroidDF(x, mf):
    sum = np.sum(mf)
    mfSum = np.sum(np.multiply(mf, x))
    return mfSum / sum

def imp_zadeh(x,y):
    return np.fmin( 1.0 , 1.0 - x + y )

def imp_cor_min(x,y):
    return np.fmin( x, y )

def imp_cor_prod(x,y):
    return np.multiply(x, y)

def implication(x,y,type):
    if (type == 0):
        return np.fmin(1.0, 1.0 - x + y)
    elif (type == 1):
        return np.fmin(x, y)
    elif (type == 2):
        return np.multiply(x, y)

# interpret membership
def interpMem(values, memFunc, x):
    lIndex = None
    for i in range(len(values)):
        if values[i] > x:
            lIndex = i - 1
            break
    
    if lIndex is None: #if the index is at the end of the array
        lIndex = len(values) - 2 # set to the second to last index 

    left = memFunc[lIndex]
    right = memFunc[lIndex + 1]

    leftWeight = (values[lIndex + 1] - x) / (values[lIndex + 1] - values[lIndex])
    rightWeight = 1 - leftWeight

    return (leftWeight * left) + (rightWeight * right)
    
def aggregate(rules, type):
    if (type == 0):
        return np.fmax.reduce(rules)
    if (type == 1):
        return sum(rules)

In [3]:
# fuzzy membership functions
x_sl = np.arange(4, 8.1, .1)
x_sw = np.arange(2, 5.1, .1)
x_pl = np.arange(1, 7.1, .1)
x_pw = np.arange(0, 3.1, .1)

sl_lo = trigMem(x_sl, [4, 4, 6])
sl_md = trigMem(x_sl, [5, 6, 7])
sl_hi = trigMem(x_sl, [6, 8, 8])

sw_lo = trigMem(x_sw, [2, 2, 3.5])
sw_md = trigMem(x_sw, [2.5, 3.5, 4.5])
sw_hi = trigMem(x_sw, [3.5, 5, 5])

pl_lo = trigMem(x_pl, [1, 1, 4])
pl_md = trigMem(x_pl, [2.5, 4, 5.5])
pl_hi = trigMem(x_pl, [4, 7, 7])

pw_lo = trigMem(x_pw, [0, 0, 1.5])
pw_md = trigMem(x_pw, [0.5, 1.5, 2.5])
pw_hi = trigMem(x_pw, [1.5, 3, 3])


x_species = np.arange(0, 3.1, 0.1)
s_setosa = trigMem(x_species, [0, 0, 2])
s_versi = trigMem(x_species, [0.5, 1.5, 2.5])
s_virgi = trigMem(x_species, [1, 3, 3])

In [4]:
# 0 - Lukasiewicz
# 1 - Correlation Min
# 2 - Correlation Product
impOp = 2

# 0 - Maximum
# 1 - Sum
aggOp = 0

In [5]:
def evaluateIndividual(ind):
    sl_level_lo = interpMem(x_sl, sl_lo, ind[0])
    sl_level_md = interpMem(x_sl, sl_md, ind[0])
    sl_level_hi = interpMem(x_sl, sl_hi, ind[0])

    sw_level_lo = interpMem(x_sw, sw_lo, ind[1])
    sw_level_md = interpMem(x_sw, sw_md, ind[1])
    sw_level_hi = interpMem(x_sw, sw_hi, ind[1])

    pl_level_lo = interpMem(x_pl, pl_lo, ind[2])
    pl_level_md = interpMem(x_pl, pl_md, ind[2])
    pl_level_hi = interpMem(x_pl, pl_hi, ind[2])

    pw_level_lo = interpMem(x_pw, pw_lo, ind[3])
    pw_level_md = interpMem(x_pw, pw_md, ind[3])
    pw_level_hi = interpMem(x_pw, pw_hi, ind[3])

    rules = []
    
    # if petal length and petal width is small then setosa
    tempRule = implication(pl_level_lo, pw_level_lo, impOp)
    rules.append(implication(tempRule, s_setosa, impOp))

    # if petal length is medium then versicolor
    rules.append(implication(pl_level_md, s_versi, impOp))

    # if sepal width is small then versicolor
    # rules.append(implication(sw_level_lo, s_versi, impOp)) - bad rule

    # if petal length is big then virginica
    rules.append(implication(pl_level_hi, s_virgi, impOp))

    # if sepal length is big then virginica
    # rules.append(implication(sl_level_hi, s_virgi, impOp)) - bad rule
    



    aggregated = aggregate(rules, aggOp)
    # print(x_species)
    # print(aggregated)
    species = centroidDF(x_species, aggregated)
    # print(species)
    # species_activation = interpMem(x_species, aggregated, species)

    return int(species)


In [6]:
correct = 0

for index, individual in enumerate(iris.data):
    result = evaluateIndividual(individual)
    if (result == iris.target[index]):
        correct += 1
    else:
        print("incorrect classification")
        print("individual was: ", individual)
        print("classified incorrectly as: ", result)
        print("should have been: ", iris.target[index])

correctRate = correct / len(iris.data)
print(correctRate)
    

incorrect classification
individual was:  [6.  2.7 5.1 1.6]
classified incorrectly as:  2
should have been:  1
incorrect classification
individual was:  [4.9 2.5 4.5 1.7]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [5.7 2.5 5.  2. ]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [6.  2.2 5.  1.5]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [5.6 2.8 4.9 2. ]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [6.3 2.7 4.9 1.8]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [6.2 2.8 4.8 1.8]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [6.1 3.  4.9 1.8]
classified incorrectly as:  1
should have been:  2
incorrect classification
individual was:  [6.  3.  4.8 1.8]
classified incorrectly as:  1
should have been:  2
i